In [ ]:
# !pip install pypdf
# !pip install -U transformers langchain langchain_huggingface langchain_community sentence-transformers langchain_huggingface langchain_core chromadb 

In [4]:
# !pip install ipywidgets
# !pip install langchain_huggingface

In [5]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [6]:
pdf_directory = os.getcwd() + "/pdf_files"
print(pdf_directory)

/home/jovyan/work/pdf_files


In [7]:
def load_pdf_files(directory):
    documents = []
    # Loop ผ่านไฟล์ทั้งหมดใน directory
    for filename in os.listdir(directory):
        if filename.endswith('.pdf'):
            file_path = os.path.join(directory, filename)
            try:
                # โหลด PDF โดยใช้ PyPDFLoader
                loader = PyPDFLoader(file_path)
                docs = loader.load()
                documents.extend(docs)
                print(f"Loaded: {filename}")
            except Exception as e:
                print(f"Error loading {filename}: {str(e)}")
    return documents

In [8]:
# 3. โหลดเอกสารทั้งหมด
all_documents = load_pdf_files(pdf_directory)

Loaded: C-Outline_Faciltate-Virtual-Team.pdf
Loaded: KPI-5.1.2.3-320781.pdf
Loaded: หลักสูตร_1415863499.pdf
Loaded: Course Outline e-Learning (หลักสูตร).pdf
Loaded: SW201 ภาค  2-2563 (All_Gr.).pdf
Loaded: Training outline_500026_0.pdf
Loaded: C16-สร้างทีมงานDISC-Model.pdf


In [9]:
# 4. แบ่งเอกสารเป็นส่วนย่อยๆ (chunking)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,  # ขนาดของแต่ละ chunk
    chunk_overlap=54,  # การ overlap ระหว่าง chunk
    length_function=len,
    strip_whitespace=True,
)
split_docs = text_splitter.split_documents(all_documents)
print(f"Total chunks created: {len(split_docs)}")

Total chunks created: 154


In [13]:
import os
from typing import List
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document  # นำเข้า Document

def ingest(chunks: List[Document], embeddings_dir:str):
    """
    ฟังก์ชันที่รับรายการของ Document (chunks) และนำไปสร้าง Vector database
    """

    model_name = "BAAI/bge-m3"
    model_kwargs = {'device': 'cpu'}
    encode_kwargs = {'normalize_embeddings': False}
    
    embedding = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    
   # ตรวจสอบว่ามี vector store อยู่แล้วหรือไม่
    if os.path.exists(os.path.join(embeddings_dir, "chroma.sqlite3")):
        print("--append data to vector store--")
        # โหลด vector store ที่มีอยู่
        vector_store = Chroma(persist_directory=embeddings_dir, embedding_function=embedding)
        # ลบ documents ทั้งหมดก่อน
        vector_store.delete_collection()
        # สร้าง collection ใหม่โดยอัตโนมัติเมื่อเพิ่ม documents
        vector_store.add_documents(documents=chunks)
    else:
        print("--new vector store--")
        # สร้างไดเรกทอรีสำหรับบันทึก embeddings
        os.makedirs(embeddings_dir, exist_ok=True)
        # สร้าง vector store ใหม่และบันทึก embeddings ลงในไฟล์
        Chroma.from_documents(documents=chunks, embedding=embedding, persist_directory=embeddings_dir)

    print("--- Ingest to Vector Database Success ---")


In [12]:
ingest(chunks=split_docs, embeddings_dir="chroma-pdfs")

tokenizer.json:  61%|######1   | 10.5M/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

model.safetensors:   3%|2         | 62.9M/2.27G [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

--new vector store--
--- Ingest to Vector Database Success ---


In [18]:
embeddings_dir="chroma-pdfs"

model_name = "BAAI/bge-m3"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
vector_store = Chroma(persist_directory=embeddings_dir, embedding_function=embedding)
print("-- Vector Store Loaded --")

-- Vector Store Loaded --


In [19]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})
print("--retriever--")

--retriever--


In [21]:
def genPrompt( question: str, retriever: BaseRetriever):
        """
        Generate a prompt based on the retrieved documents for the given question.
        """
        retrieved_docs = retriever.invoke(question)
        context = ' '.join([doc.page_content for doc in retrieved_docs])
        prompt = f"""
            [Instructions] 
                Question: {question}
                Context: {context} 
                Answer:
            [/Instructions]
            """
        return prompt

In [25]:
prompt = genPrompt(question="หลักสูตรผู้บริหาร",retriever=retriever)
print(prompt)


            [Instructions] 
                Question: หลักสูตรผู้บริหาร
                Context: 1 
 
หลักสูตร 
การจัดการ 
เรียนรู้ 
 การวัดและ 
 ประเมินผล 
 
การ 
สะท้อนผล 
หลักสูตร  (Curriculum) 
 
รองศาสตราจารย์ ดร.วิชัย วงษ์ใหญ่* 
 
 
ความหมายของหลักสูตร 
 หลักสูตร (Curriculum) เป็นค าที่มีรากศัพท์มาจากค าในภาษาลาตินว่า “race – course” 
ซึ่งหมายถึง เส้นทางที่ใช้วิ่งแข่งขัน  เนื่องมาจากเป้าหมายของหลักสูตรที่มุ่งหวังให้ผู้เรียนสามา รถ
เจริญเติบโตเป็นผู้ใหญ่ ที่มีคุณภาพและประสบความส าเร็จในการด ารงชีวิตอยู่ในสังคมแห่งอนาคต ซึ่งจะเป็นข้อมูลสารสนเทศส าหรับการปรับปรุงหลักสูตรต่อไปดังแผนภาพ  1 
 
 
 
 
 
 
 
แผนภาพ  1   ความส าคัญของหลักสูตรที่ส่งผลต่อการจัดการเรียนรู้  การวัดและประเมินผล 
     และการสะท้อนผลเพื่อปรับปรุงหลักสูตร 
* สาขาการวิจัยและพัฒนาหลักสูตร บัณฑิตวิทยาลัย มหาวิทยาลัยศรีนครินทรวิโรฒ ระบบ 
การพัฒนาหลักสูตร 
ระบบ 
การปรับปรุงหลักสูตร  
ระบบ 
การประเมินหลักสูตร 
 
ระบบ 
การใช้หลักสูตร 
 
 
 
 
 
 
 
แผนภาพ  2 ความสัมพันธ์ระหว่างระบบการพัฒนาหลักสูตร  ระบบการใช้หลักสูตร   